In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os, cv2

!pip install efficientnet

     |████████████████████████████████| 51kB 2.2MB/s 


In [ ]:
# FILE STRUCTURE

train_path = '/content/drive/My Drive/Data/DashSpeed/Train/Train/'
val_path = '/content/drive/My Drive/Data/DashSpeed/Validation/Validation/'
test_path = '/content/drive/My Drive/Data/DashSpeed/Test/Test/'

train_ids = os.listdir(train_path)
val_ids = os.listdir(val_path)
test_ids = os.listdir(test_path)

img_names = [f'00{i}.jpg' for i in range(1,9)]

IMAGE_SHAPE = (720//2,1280//2,3)

In [ ]:
import keras.applications as apn
import efficientnet.keras as efn
from keras.models import Model
from keras.layers import *

base = apn.InceptionResNetV2(weights='imagenet', input_shape=IMAGE_SHAPE, include_top=False, pooling='avg')
base.trainable=False

def generate_features(data_path, data_ids, model, dst_path, type_of_data):
    npy_path = []
    speeds = []
    iter = 0
    for id in data_ids:
        print(iter, end=' ')
        iter += 1
        eight_images = []

        for img_name in img_names:
            img = plt.imread(fname = data_path + id + '/imgs/' + img_name)
            img = cv2.resize(img, (720//2, 1280//2)).reshape(1,IMAGE_SHAPE[0],IMAGE_SHAPE[1],IMAGE_SHAPE[2])
            npy = model.predict(img)
            eight_images.append(npy)
        
        features = np.array(eight_images)
        
        feature_file = f'{id}.npy'

        if type_of_data != 'Test':
            x,y = pd.read_json(data_path + id + '/annotation.json').velocity.values[0]
            speed = np.sqrt(x**2 + y**2)
            npy_path.append(feature_file)
            speeds.append(speed)
        
        else:
            speed = np.nan
            npy_path.append(feature_file)
            speeds.append(speed)
        
        np.save(file=f'{dst_path}/{type_of_data}/{id}.npy', arr=features)

    dataframe = pd.DataFrame({'id':npy_path, 'speed':speeds})
    dataframe.to_csv(f'{dst_path}/{type_of_data}_features.csv', index=False)
    

print('Train:')
generate_features(data_path=train_path, data_ids=train_ids, model=base, dst_path='/content/drive/My Drive/Data/DashSpeed/FeaturesDenseOriginal',
                  type_of_data='Train')
print()
print('Test:')
generate_features(data_path=test_path, data_ids=test_ids, model=base, dst_path='/content/drive/My Drive/Data/DashSpeed/FeaturesDenseOriginal',
                  type_of_data='Test')
print()
print('Validation:')
df = generate_features(data_path=val_path, data_ids=val_ids, model=base, dst_path='/content/drive/My Drive/Data/DashSpeed/FeaturesDenseOriginal',
                  type_of_data='Validation')